In [1]:
# Utility
import re, sys, copy, arrow, random
from helpers import *

In [ ]:
goal = int(get_raw(2015, 20))
print(goal)

def should_give(elf, house):
    return house % elf == 0

def calc_presents(house):
    rv = 0
    for elf in range(1, house+1):
        if should_give(elf, house):
            rv += (elf * 10)
    return rv

house = 0
while True:
    house += 1
    
    if calc_presents(house) >= goal:
        print(house)
        break

In [22]:
@timeit
def day19():
    lines = get_lines(2015, 19)
    swaps = []
    flipped = {}
    stop = False
    molecule = ""

    for line in lines:
        if line == "":
            stop = True
            continue
        if stop is False:
            frm, to = line.split(' => ')
            frm, to = frm.strip(), to.strip()
            swaps.append((frm, to))
            flipped[to[::-1]] = frm[::-1]
        if stop is True:
            molecule = line

    def perms():
        for frm, to in swaps:
            for match in re.finditer(frm, molecule):
                s, e = match.span()
                yield molecule[0:s] + to + molecule[e:]

    def steps(seed):
        re_doit = re.compile("|".join(flipped.keys()))
        def rep(x):
            return flipped[x.group()]
        steps = 0
        mol = seed[::-1]
        while mol != 'e':
            mol = re_doit.sub(rep, mol, 1)
            steps +=1
        return steps

    uniques = set()
    [uniques.add(p) for p in perms()]
    handle_result(len(uniques), 2015, 19, "a")
    handle_result(steps(molecule), 2015, 19, "b")

day19()

576
Part a already solved with same answer: 576
207
Part b already solved with same answer: 207
day19 7.31 ms


In [21]:
@timeit
def day18():
    from helpers.grids.rc import data_to_grid, grid_print, grid_yield

    def update(current, corners=None):
        rv = {}
        for r, c, eol in grid_yield(current):
            if eol: continue
            pt = (r,c)
            nextdoor = [current.get(n, ".") for n in neighbors8(pt)]
            nextdoor_on_count = sum([1 for i in nextdoor if i == "#"])
            if current[pt] == "#":
                rv[pt] = "#" if nextdoor_on_count == 2 or nextdoor_on_count == 3 else "."
            else:
                rv[pt] = "#" if nextdoor_on_count == 3 else "."
        if corners:
            for corner in corners:
                rv[corner] = "#"
        return rv

    def run(corners_on=False):
        grid, hx, hy = data_to_grid(get_lines(2015, 18))
        corners = None if not corners_on else [(0,0), (hx, hy), (0, hy), (hx, 0)]

        if corners_on:
            for corner in corners:
                grid[corner] = "#"

        for _ in range(100):
            grid = update(grid, corners)

        cnt = 0
        for r, c, eol in grid_yield(grid):
            if eol: continue
            cnt += 1 if grid[(r,c)] == "#" else 0
        return cnt

    handle_result(run(), 2015, 18, "a")
    handle_result(run(True), 2015, 18, "b")

day18()

814
Part a already solved with same answer: 814
924
Part b already solved with same answer: 924
day18 6157.98 ms


In [18]:
@timeit
def day17():
    lines = [int(l) for l in get_lines(2015, 17)]
    counter = []
    for c in powerset(lines):
        if sum(c) == 150:
            counter.append(len(c))
    handle_result(len(counter), 2015, 17, "a")
    smallest = min(counter)
    all_small = [i for i in counter if i == smallest]
    handle_result(len(all_small), 2015, 17, "b")

day17()

1304
Part a already solved with same answer: 1304
18
Part b already solved with same answer: 18
day17 308.79 ms


In [17]:
@timeit
def day16():
    lines = get_lines(2015, 16)
    sue = {}

    def to_dict(line):
        rv = {}
        pieces = line.split(', ')
        for piece in pieces:
            parts = piece.split(': ')
            rv[parts[0].strip()] = int(parts[1].strip())
        return rv

    proper = { 'children': 3, 'cats': 7, 'samoyeds': 2, 'pomeranians': 3, 'akitas': 0, 'vizslas': 0, 'goldfish': 5, 'trees': 3, 'cars': 2, 'perfumes': 1 }

    def check1(entry):
        for name, value in proper.items():
            if entry.get(name, None) is not None and entry[name] != value: return False
        return True

    greater = set(['cats', 'trees'])
    lesser = set(['pomeranians', 'goldfish'])
    def check2(entry):
        for name, value in proper.items():
            if entry.get(name, None) is not None:
                if name in greater and entry[name] <= value: return False
                if name in lesser and entry[name] >= value: return False
                if name not in greater and name not in lesser and entry[name] != value: return False
        return True

    for line in lines:
        name, datum = line.split(':', 1)
        data = to_dict(datum)
        if check1(data):
            handle_result(get_ints(name)[0], 2015, 16, "a")
        if check2(data):
            handle_result(get_ints(name)[0], 2015, 16, "b")

day16()

260
Part b already solved with same answer: 260
373
Part a already solved with same answer: 373
day16 6.09 ms


In [16]:
@timeit
def day15():
    raw = get_lines(2015, 15)
    re_entry = re.compile(r"(?P<name>.*?): capacity (?P<capacity>[-\d]+?), durability (?P<durability>[-\d]+?), flavor (?P<flavor>[-\d]+?), texture (?P<texture>[-\d]+?), calories (?P<calories>[-\d]+?)")
    data = [re_entry.search(l).groupdict() for l in raw]
    for d in data:
        for k, v in d.items():
            if k != 'name':
                d[k] = int(v)

    def get_prop(name, combos):
        rv = 0
        for idx, m in enumerate(combos):
            rv += data[idx][name] * m
        return 0 if rv < 0 else rv

    nums = list(range(1,100))
    max_score = 0
    max_combo = None
    max_calorie_score = 0
    max_calorie_combo = None
    for c in list(permutations(nums, len(data))):
        if sum(c) != 100: continue
        score = get_prop('capacity', c) * get_prop('durability', c) * get_prop('flavor', c) * get_prop('texture', c)
        if score > max_score:
            max_score = score
            max_combo = c
        if get_prop('calories', c) == 500 and score > max_calorie_score:
            max_calorie_score = score
            max_calorie_combo = c
    handle_result(max_score, 2015, 15, "a")
    handle_result(max_calorie_score, 2015, 15, "b")

day15()

18965440
Part a already solved with same answer: 18965440
15862900
Part b already solved with same answer: 15862900
day15 36610.38 ms


In [15]:
@timeit
def day14():
    
    @dataclass
    class Deer:
        name: str
        speed: int
        time: int
        rest: int
        f: int = None
        r: int = None
        d: int = None
        s: int = None

        def start(self):
            self.f = self.time
            self.r = None
            self.d = 0
            self.s = 0

        def tick(self):
            if self.f is not None:
                self.d += self.speed
                self.f -= 1
                if self.f == 0:
                    self.f = None
                    self.r = self.rest
                return
            if self.r is not None:
                self.r -= 1
                if self.r == 0:
                    self.r = None
                    self.f = self.time

        def score(self):
            self.s += 1

    def make_deer(item):
        return Deer(name=item[0], speed=int(item[1]), time=int(item[2]), rest=int(item[3]))

    def leaders(deers):
        first_lead = max(deers, key=lambda d: d.d)
        return [d for d in deers if d.d == first_lead.d]

    raw = get_lines(2015, 14)
    re_entry = re.compile(r"(.*?) can fly (\d+) km/s for (\d+) seconds, but then must rest for (\d+) seconds.")
    deers = [make_deer(re_entry.findall(l)[0]) for l in raw]
    [d.start() for d in deers]
    for i in range(2503):
        [d.tick() for d in deers]
        [d.score() for d in leaders(deers)]

    distance = max(deers, key=lambda d: d.d)
    handle_result(distance.d, 2015, 14, "a")

    points = max(deers, key=lambda d: d.s)
    handle_result(points.s, 2015, 14, "b")
    
day14()

2655
Part a already solved with same answer: 2655
1059
Part b already solved with same answer: 1059
day14 24.35 ms


In [14]:

@timeit
def day13():
    raw = get_lines(2015, 13)
    re_entry = re.compile(r"(.*?) would (lose|gain) (\d+) happiness units by sitting next to (.*)\.")
    data = [re_entry.findall(l)[0] for l in raw]

    relationships = defaultdict(int)
    names = set()
    for datum in data:
        names.add(datum[0])
        names.add(datum[3])
        key = (datum[0], datum[3])
        value = int(datum[2]) * (-1 if datum[1] == "lose" else 1)
        relationships[key] = value
    
    def find_best():
        best = 0
        for perm in permutations(names):
            rv = relationships[(perm[0], perm[-1])] + relationships[(perm[-1], perm[0])]
            for pair in pairwise(perm):
                rv += relationships[(pair[0], pair[1])] + relationships[(pair[1], pair[0])]
            best = max(best, rv)
        return best

    handle_result(find_best(), 2015, 13, "a")
    
    for name in names:
        relationships[("me", name)] = 0
        relationships[(name, "me")] = 0
    names.add("me")

    handle_result(find_best(), 2015, 13, "b")

day13()

618
Part a already solved with same answer: 618
601
Part b already solved with same answer: 601
day13 2637.25 ms


In [13]:
@timeit
def day12():
    data = get_raw(2015, 12)
    
    def dict_value(item):
        rv = 0
        for v in item.values():
            iv = item_value(v)
            if iv is None: return 0
            rv += iv
        return rv

    def list_value(item):
        rv = 0
        for v in item:
            iv = item_value(v)
            rv += 0 if iv is None else iv
        return rv

    def item_value(item):
        if isinstance(item, str) and item == "red": return None
        if isinstance(item, str): return 0
        if isinstance(item, int): return item
        if isinstance(item, list): return list_value(item)
        if isinstance(item, dict): return dict_value(item)
    
    def part1():
        ints = get_ints(data)
        return sum(ints)
    handle_result(part1(), 2015, 12, "a")
    
    def part2():
        import json
        doc = json.loads(data)
        return item_value(doc)
    handle_result(part2(), 2015, 12, "b")
day12()

111754
Part a already solved with same answer: 111754
65402
Part b already solved with same answer: 65402
day12 9.31 ms


In [12]:
@timeit
def day11():
    current = get_raw(2015, 11)
    re_pairs = re.compile(r"(.)\1")
    ord_z = ord('z')
    
    def roll_char(char):
        ord_char = ord(char) + 1
        if ord_char > ord_z:
            return 'a', True
        return chr(ord_char), False
    
    def roll_password(seed):
        current = seed
        while True:
            backwards = current[::-1]
            rv = ""
            overflow = True
            for ch in backwards:
                if overflow is False:
                    rv += ch
                else:
                    new_ch, overflow = roll_char(ch)
                    rv += new_ch
            current = rv[::-1]
            yield current
            
    def check1(potential):
        if "i" in potential or "o" in potential or "l" in potential: return False
        pairs = re_pairs.findall(potential)
        if len(pairs) < 2: return False
        for c1, c2, c3 in overlapping(potential, 3):
            if ord(c2) - ord(c1) == 1 and ord(c3) - ord(c2) == 1: return True
        return False
    
    def find_next(current):
        for pw in roll_password(current):
            if check1(pw):
                return pw
            
    first = find_next(current)
    handle_result(first, 2015, 11, "a")
    second = find_next(first)
    handle_result(second, 2015, 11, "b")
    
        
day11()

cqjxxyzz
Part a already solved with same answer: cqjxxyzz
cqkaabcc
Part b already solved with same answer: cqkaabcc
day11 1780.75 ms


In [11]:
@timeit
def day10():
    def las(orig):
        rv = ""
        cur = None
        cnt = 0
        for ch in orig:
            if ch != cur:
                if cur is not None:
                    rv += f"{cnt}{cur}"
                cur = ch
                cnt = 1
            else:
                cnt += 1
        rv += f"{cnt}{cur}"
        return rv
    
    def run(seed, times):
        current = seed
        for i in range(times):
            current = las(current)
        return len(current)
    
    seed = get_raw(2015, 10)
    handle_result(run(seed, 40), 2015, 10, "a")
    handle_result(run(seed, 50), 2015, 10, "b")

day10()

252594
Part a already solved with same answer: 252594
3579328
Part b already solved with same answer: 3579328
day10 2637.35 ms


In [10]:
@timeit
def day9():
    def process_line(line):
        pieces = line.split(' ')
        return (pieces[0], pieces[2], int(pieces[-1]))
    
    lines = [process_line(l) for l in get_lines(2015, 9)]
    cities = set()
    distances = defaultdict(int)
    for s, e, d in lines:
        cities.add(s)
        cities.add(e)
        distances[(s, e)] = d
        distances[(e, s)] = d
        
    def naive(func):
        rv_dist = None
        for p in permutations(cities):
            dist = 0
            for c1, c2 in pairwise(p):
                dist += distances[(c1, c2)]
            rv_dist = dist if rv_dist is None else func(rv_dist, dist)
        return rv_dist
    
    handle_result(naive(min), 2015, 9, "a")
    handle_result(naive(max), 2015, 9, "b")

day9()

251
Part a already solved with same answer: 251
898
Part b already solved with same answer: 898
day9 347.97 ms


In [9]:
@timeit
def day8():
    lines = get_lines(2015, 8)
    re_hex = re.compile(r"\\x[0-9a-f]{2}")
    re_slash = re.compile(r"\\\\")
    re_quote = re.compile(r"\\\"")

    def decode(s):
        rv = re_hex.sub("!", s)
        rv = re_slash.sub("!", rv)
        rv = re_quote.sub("!", rv)
        return rv

    def encode(s):
        rv = ""
        for ch in s:
            if ch == "\\" or ch == "\"":
                rv += ch + ch
            else:
                rv += ch
        return rv

    orig = decoded = encoded = 0
    for line in lines:
        orig += len(line)
        decoded += len(decode(line)) - 2
        encoded += len(encode(line)) + 2

    handle_result(orig-decoded, 2015, 8, "a")
    handle_result(encoded-orig, 2015, 8, "b")

day8()

1371
Part a already solved with same answer: 1371
2117
Part b already solved with same answer: 2117
day8 6.50 ms


In [8]:
@timeit
def day7():
    lines = get_lines(2015, 7)
    re_operation = re.compile(r"NOT (.*)|(.*?) AND (.*)|(.*?) OR (.*)|(.*?) LSHIFT (\d+)|(.*?) RSHIFT (\d+)")
    re_op = re.compile(r"NOT|AND|OR|LSHIFT|RSHIFT")

    @dataclass
    class Wire:
        name: str
        raw: str
        op: str
        value: int
        upstream: list[int]
        shift: int
        orig: int

    commands = {}
    for line in lines:
        cmd, name = line.split(' -> ')
        wire = Wire(name=name, raw=cmd, op=None, value=None, upstream=None, shift=None, orig=None)
        match = re_operation.search(cmd)
        if match is None:
            if cmd.isnumeric():
                wire.value = int(cmd) # hard-coded value
                wire.orig = wire.value
            else:
                wire.op = "SET"
                wire.upstream = [cmd] # simple value from upstream
        else:
            wire.op = re_op.findall(cmd)[0]
            wire.upstream = [g for g in match.groups() if g is not None]
            if "SHIFT" in wire.op:
                shifts = [int(g) for g in match.groups() if g is not None and g.isnumeric() is True]
                if len(shifts) == 1:
                    wire.shift = shifts[0]        
        commands[name] = wire

    def settle():
        rv = 0
        while True:
            rv += 1
            stop = True
            for wire in commands.values():
                if wire.value is not None: continue

                upstream_values = []
                for u_name_or_value in wire.upstream:
                    if u_name_or_value.isnumeric():
                        upstream_values.append(int(u_name_or_value))
                    else:
                        u_value = commands[u_name_or_value].value
                        if u_value is not None:
                            upstream_values.append(u_value)
                if len(upstream_values) < len(wire.upstream):
                    stop = False
                    continue

                if wire.op == "SET": wire.value = upstream_values[0]
                if wire.op == "AND": wire.value = upstream_values[0] & upstream_values[1]
                if wire.op == "OR": wire.value = upstream_values[0] | upstream_values[1]
                if wire.op == "NOT": wire.value = ~ upstream_values[0]
                if wire.op == "LSHIFT": wire.value = upstream_values[0] << wire.shift
                if wire.op == "RSHIFT": wire.value = upstream_values[0] >> wire.shift

            if stop is True: return rv

    loops_1 = settle()
    value_of_a = commands['a'].value
    handle_result(value_of_a, 2015, 7, "a")

    for wire in commands.values():
        wire.value = wire.orig

    commands['b'].value = value_of_a
    loops_2 = settle()
    handle_result(commands['a'].value, 2015, 7, "b")

day7()

956
Part a already solved with same answer: 956
40149
Part b already solved with same answer: 40149
day7 38.03 ms


In [7]:
@timeit
def day6():
    lines = get_lines(2015, 6)
    
    def loop(func):
        lights = defaultdict(int)
        for line in lines:
            sx, sy, ex, ey = get_ints(line)
            for x in range(sx, ex+1):
                for y in range(sy, ey+1):
                    pt = (x, y)
                    lights[pt] = func(line, lights[pt])
        rv = 0
        for k, v in lights.items():
            rv += v
        return rv
    
    def do_cmd1(cmd, current):
        if "turn on" in cmd: return 1
        if "turn off" in cmd: return 0
        return current ^ 1
    
    def do_cmd2(cmd, current):
        if "turn on" in cmd: return current + 1
        if "turn off" in cmd: return 0 if current - 1 <= 0 else current - 1
        return current + 2
    
    handle_result(loop(do_cmd1), 2015, 6, "a")
    handle_result(loop(do_cmd2), 2015, 6, "b")
    
day6()

569999
Part a already solved with same answer: 569999
17836115
Part b already solved with same answer: 17836115
day6 22572.85 ms


In [6]:
@timeit
def day5():
    lines = get_lines(2015, 5)
    
    def loop(func):
        rv = 0
        for line in lines:
            rv += 1 if func(line) else 0
        return rv
    
    re_vowels = re.compile(r"[aeiou]")
    re_double = re.compile(r"(.)\1")
    re_bad = re.compile(r"ab|cd|pq|xy")
    
    def nice1(s):
        if len(re_vowels.findall(s)) < 3: return False
        if len(re_double.findall(s)) <= 0: return False
        if len(re_bad.findall(s)) > 0: return False
        return True
    
    re_pairs = re.compile(r"(..).*\1")
    re_repeat = re.compile(r"(.).\1")
    
    def nice2(s):
        if len(re_pairs.findall(s)) <= 0: return False
        if len(re_repeat.findall(s)) <= 0: return False
        return True
    
    handle_result(loop(nice1), 2015, 5, "a")
    handle_result(loop(nice2), 2015, 5, "b")

day5()

238
Part a already solved with same answer: 238
69
Part b already solved with same answer: 69
day5 9.54 ms


In [5]:
@timeit
def day4():
    import hashlib
    key = get_raw(2015, 4)
    
    def find(num):
        current = 0
        answer = "0" * num
        while True:
            current += 1
            value = f"{key}{current}"
            digest = hashlib.md5(value.encode('utf-8')).hexdigest()
            if (digest[:num] == answer):
                return current        
                
    handle_result(find(5), 2015, 4, "a")           
    handle_result(find(6), 2015, 4, "b")

day4()

282749
Part a already solved with same answer: 282749
9962624
Part b already solved with same answer: 9962624
day4 19075.80 ms


In [4]:
@timeit
def day3():
    from helpers.grids.xy import LEFT, RIGHT, UP, DOWN
    
    data = get_raw(2015, 3)
    DIRECTIONS = {
        '>': LEFT,
        '<': RIGHT,
        '^': UP,
        'v': DOWN
    }
    
    def part1():
        houses = defaultdict(int)
        current = origin
        houses[current] += 1
        for turn in data:
            current = add(current, DIRECTIONS[turn])
            houses[current] += 1
        return len(houses.keys())
    handle_result(part1(), 2015, 3, "a")
    
    def part2():
        houses = defaultdict(int)
        current_santa = origin
        current_bot = origin
        houses[origin] += 2
        for santa_turn, bot_turn in grouper(data, 2):
            current_santa = add(current_santa, DIRECTIONS[santa_turn])
            houses[current_santa] += 1
            current_bot = add(current_bot, DIRECTIONS[bot_turn])
            houses[current_bot] += 1
        return len(houses.keys())
    handle_result(part2(), 2015, 3, "b")
day3()

2081
Part a already solved with same answer: 2081
2341
Part b already solved with same answer: 2341
day3 29.10 ms


In [3]:
@timeit
def day2():
    def area_paper(l, w, h):
        return (2*l*w) + (2*w*h) + (2*h*l) + min([l*w, w*h, h*l])
    
    def length_ribbon(l, w, h):
        return min([l+l+w+w, w+w+h+h, h+h+l+l]) + (l*w*h)
    
    lines = [get_ints(l) for l in get_lines(2015,2)]
    
    def loop(func):
        rv = 0
        for l, w, h in lines:
            rv += func(l, w, h)
        return rv
    
    handle_result(loop(area_paper), 2015, 2, "a")
    handle_result(loop(length_ribbon), 2015, 2, "b")

day2()

1588178
Part a already solved with same answer: 1588178
3783758
Part b already solved with same answer: 3783758
day2 10.21 ms


In [2]:
@timeit
def day1():
    data = get_raw(2015, 1)
    floor = 0
    basement = False
    for idx, cmd in enumerate(data, start=1):
        floor += (1 if cmd == "(" else -1)
        if floor == -1 and basement is False:
            print(f"part 2 is {idx}")
            basement = True
    print(f"part 1 is {floor}")
    
day1()

part 2 is 1795
part 1 is 74
day1 3.05 ms
